# AI Studio resource creation

In this notebook, you will create all the necessary resources for AI Studio including:

* hub
* project
* AI Services resource
* AI Services connection to hub
* Open AI deployment

Requirements:
* Azure subscription

## Prerequisites

You'll need your Subscription Id and resource group name to configure the ML Client.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

my_resource_group = "{RESOURCE_GROUP}"
my_subscription_id = "{SUBSCRIPTION_ID}"
my_location = "East US"

ml_client = MLClient(resource_group_name=my_resource_group, subscription_id=my_subscription_id,credential=DefaultAzureCredential())

### Naming
Input your preferred named for the resources that will be created.

In [ ]:
## Hub
my_hub_name = "myexamplehub"
my_hub_display_name = "My Example Hub"

## Project
my_project_name = "myexampleproject"
my_project_location = my_location
my_project_display_name = "My Example Project"

## Connection
my_connection_name = "myaiservivce"

## Hub creation

### Create a basic hub

A basic hub will automatically create an Azure Key Vault and Azure Storage.

If you would like to include an existing Azure Key Vault and Azure Storage, make sure to add the additional fields:
* storage_account = "{existing_storage_ARM_id}"
* key_vault = "{existing_keyvault_ARM_id}"

In [ ]:
from azure.ai.ml.entities import Hub

# construct a basic hub
my_hub = Hub(name=my_hub_name, 
            location=my_location,
            display_name=my_hub_display_name)

created_hub = ml_client.workspaces.begin_create(my_hub).result()

## Project creation

A project will be created and associated to the hub we just created.

In [ ]:
from azure.ai.ml.entities import Project

# construct the project
my_project = Project(name=my_project_name, 
                location=my_project_location,
                display_name=my_project_display_name,
                hub_id=created_hub.id)

created_project = ml_client.workspaces.begin_create(workspace=my_project).result()

## AI Services Connection

Using an existing Azure AI Services resource, create a connection to the hub.

In [ ]:
from azure.ai.ml.entities import AzureAIServicesConnection

# construct an AI Services connection
my_endpoint = "demo.endpoint" # this could also be called target
my_api_keys = None # leave blank for Authentication type = AAD
my_ai_services_resource_id = "" # ARM id required

my_connection = AIServicesConnection(name=my_connection_name,
                                    endpoint=my_endpoint, 
                                    api_key= my_api_keys,
                                    ai_services_resource_id=my_ai_services_resource_id)

created_connection = ml_client.connections.create_or_update(my_connection)

## Update the ML Client

We will input the newly created project into the MLClient to use it.

In [ ]:
ml_client = MLClient(workspace_name=my_project_name, resource_group_name=my_resource_group, subscription_id=my_subscription_id,credential=DefaultAzureCredential())